In [1]:
import utility 
import numpy as np
from importlib import reload
reload(utility)
import sys
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

HOME = '/Users/amirjoudaki/Codes/string_embedding/python_tools'

file_name = 'seqs11'
data_path = HOME + '/data/' + file_name + '.npz'
result_path = HOME + '/results/' + file_name + '.ann'

Res = np.load(data_path)


seqs = Res['seqs']
vals = Res['vals']
options = Res['options']
Op = (options[()])
if int(file_name[4:])>=7:
    gene_lens = Res['gene_lens']
    num_seqs = Res['num_seqs']
else:
    num_seqs = [Op.num_seq]*Op.repeat
    

In [2]:
proj_dim = 120
k_small = 2 
k_big = 100
num_trees = 3
num_neighbours = 200


kmers, s_kmer_vals, kmer_pos, kmer_seq_id = utility.list_kmers_simple(seqs, vals = vals,  
                                         k = k_small, addy = True, padding = int(k_big))
kmer_vals = utility.get_kmver_vals(s_kmer_vals, k_big)

convolved = utility.random_projection(kmers, k_big, proj_dim)

knn_index = utility.build_index(convolved, range(len(kmers)), num_trees)
# search_indices = np.nonzero(kmer_vals>0)[0]
search_indices = [i for i in range(0,len(kmers),5)]
NN,NN_dist = utility.knn_search(knn_index, search_indices, num_neighbours)
    
quad = utility.eval_results(search_indices, NN, kmers, kmer_vals, kmer_pos, kmer_seq_id, num_seqs, options)

print('seq len = ', len(seqs[0]), ', num seqs = ', len(seqs))

print('hit ratio: ', len(quad)/len(search_indices)/num_neighbours)

computing the list of kmers and values 


100%|██████████| 10/10 [00:06<00:00,  1.70it/s]


 computing the projection


100%|██████████| 10096131/10096131 [00:11<00:00, 844780.75it/s]

 computing the convolution


...

time to build 3 trees = 236.66005086898804
nearest neighbours search:


100%|██████████| 2019227/2019227 [39:37<00:00, 849.34it/s]  

time search = 2377.425081729889



100%|██████████| 2019227/2019227 [09:19<00:00, 3611.11it/s]


evaluation time =  559.3031358718872
folls possitive =  0.9998828011009719
recall percentage  0.623
seq len =  1009224 , num seqs =  10
hit ratio:  9.758437263368607e-05


In [3]:
# print(Op)